
## Prepare Download dataset

Load credential from drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Make directory in local instance

In [2]:
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Download dataset from kaggle

In [3]:
!kaggle datasets download sriramr/fruits-fresh-and-rotten-for-classification

 99% 3.55G/3.58G [00:22<00:00, 163MB/s]
100% 3.58G/3.58G [00:22<00:00, 168MB/s]


unzip dataset

In [4]:
!unzip -q /content/fruits-fresh-and-rotten-for-classification.zip

Import necessary library

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

check gpu used

In [21]:
!nvidia-smi

Mon May 16 03:28:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    28W /  70W |   4842MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

mapping directory

In [7]:
train_dir = "/content/dataset/train"
test_dir = "/content/dataset/test"

In [19]:
tf.keras.backend.clear_session()

Add global variable

In [8]:
NUM_BATCH = 128
NUM_EPOCH = 10
SIZE_IMG = (150, 150)
DIM_IMG = (150, 150, 3)

Add Geneator and image augmentation

In [9]:
train_set = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode = 'nearest',
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,
)

val_set = ImageDataGenerator(
    rescale = 1./255,
    validation_split=0.2,
)

In [10]:
train_data = train_set.flow_from_directory(
    train_dir, 
    target_size =SIZE_IMG,
    shuffle = True,
    class_mode='categorical',
    subset = 'training',
    batch_size=128,
    seed = 128,
)

val_data = val_set.flow_from_directory(
    train_dir, 
    target_size = SIZE_IMG,
    shuffle = True,
    class_mode='categorical',
    subset = 'validation',
    batch_size=128,
    seed = 128,
)

Found 8723 images belonging to 6 classes.
Found 2178 images belonging to 6 classes.


Load pre-trained model

In [27]:
input_dim = tf.keras.Input(shape=DIM_IMG)
eff_b2_mod = tf.keras.applications.EfficientNetB0(include_top = False,
                                                weights="imagenet",
                                                  # input_tensor=input_dim,
                                                  input_shape=DIM_IMG
                                                 )

In [28]:
eff_b2_mod.trainable = True

set_trainable = False
for layer in eff_b2_mod.layers:
    if layer.name == 'multiply_16':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

Menyusun neural network layer

In [29]:
from keras.layers import Flatten, Dense, Dropout,GlobalMaxPooling2D
from keras import regularizers

model = tf.keras.models.Sequential([
#                                     layers.Conv2D(64, (3,3), activation='relu', input_shape=(300, 300, 3)),
#                                     layers.MaxPooling2D(2,2),
#                                     layers.Conv2D(32, (3,3), activation='relu'),
#                                     layers.MaxPooling2D(2,2),
                                    eff_b2_mod,
                                    GlobalMaxPooling2D(),
                                    Dense(128, activation='relu',),
                                    Dropout(0.3),
                                    Dense(32, activation='relu',),
                                    Dropout(0.3),
                                    Dense(6, activation='softmax')

])
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, metrics=['accuracy'], loss=tf.keras.losses.CategoricalCrossentropy())

Membuat callback

In [13]:
class theCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy') > 0.86):
            print("\n Akurasi mencapai target, stop training")
            self.model.stop_training = True

# Instantiate a callback object
callbacks = theCallback()

Summary and compile

In [25]:
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 5, 5, 1280)       4049571   
                                                                 
 global_max_pooling2d (Globa  (None, 1280)             0         
 lMaxPooling2D)                                                  
                                                                 
 dense (Dense)               (None, 128)               163968    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 32)                4128      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                        

In [26]:
hist = model.fit(train_data, 
                 epochs=5, 
                 validation_data=val_data, 
                #  steps_per_epoch=128, 
                 callbacks=[callbacks])

Epoch 1/5
69/69 [==============================] - 89s 1s/step - loss: 1.9292 - accuracy: 0.1804 - val_loss: 1.7878 - val_accuracy: 0.2149
Epoch 2/5
69/69 [==============================] - 79s 1s/step - loss: 1.8017 - accuracy: 0.1924 - val_loss: 1.7818 - val_accuracy: 0.2149
Epoch 3/5
69/69 [==============================] - 79s 1s/step - loss: 1.7858 - accuracy: 0.2078 - val_loss: 1.7822 - val_accuracy: 0.2149
Epoch 4/5
69/69 [==============================] - 80s 1s/step - loss: 1.7806 - accuracy: 0.2151 - val_loss: 1.7786 - val_accuracy: 0.2149
Epoch 5/5
 6/69 [=>............................] - ETA: 52s - loss: 1.7746 - accuracy: 0.2276

KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt
plt.plot(hist.history['accuracy'], label='Accuracy')
plt.plot(hist.history['loss'], label='Loss')
plt.title('Accuracy')
plt.ylabel('value')
plt.xlabel('epoch')
plt.legend()
plt.show()
# summarize history for loss
plt.plot(hist.history['val_accuracy'], label='Val Accuracy')
plt.plot(hist.history['val_loss'], label='Val Los')
plt.title('Loss')
plt.ylabel('value')
plt.xlabel('epoch')
plt.legend()
plt.show()